# 🔍 Omni-Scanner Pro v2 — 全市場形態掃描系統

| 功能 | 說明 |
|------|------|
| 台股數據 | FinMind REST API (全上市 + 上櫃 ~3,600 檔) |
| 美股數據 | Tiingo REST API (全 NYSE + NASDAQ ~7,000+ 活躍股) |
| 快取 | 12 小時檔案快取 (首次掃描後大幅加速) |
| 評分 | 幾何 50% + 量能 20% + OB 15% + Fibo 15% |
| 門檻 | 只顯示 ≥ 75 分 |
| 掃描模式 | ⚡ 快速 (~120 檔) / 🌐 全市場 (全上市公司) |

In [ ]:
#@title 📦 Step 1: 安裝套件 (不動 Colab 預裝環境)

# ⚠️ 關鍵：不卸載、不升級 numpy / scipy / pandas
# Colab 預裝版本已經夠用，動它們會造成連鎖衝突
# requests / numpy / pandas / scipy / ipywidgets 均為 Colab 預裝
!pip install plotly tqdm yfinance --quiet 2>/dev/null

print('✅ 安裝完成')

In [ ]:
#@title 📥 Step 2: 環境檢查

import importlib, sys

def check(name):
    try:
        m = importlib.import_module(name)
        v = getattr(m, '__version__', '?')
        print(f'  ✅ {name} ({v})')
        return True
    except ImportError:
        print(f'  ❌ {name}')
        return False

print('📋 環境檢查:')
print(f'  Python {sys.version.split()[0]}')
check('numpy')
check('pandas')
check('scipy')
check('plotly')
check('requests')
check('tqdm')
check('ipywidgets')
check('yfinance')

In [ ]:
#@title ⚙️ Step 3: 智慧型 K 線形態辨識引擎 (預判型)

# ============================================================
# 套件匯入
# ============================================================

import os, sys, time, json, warnings, io, zipfile, csv, math
from pathlib import Path
from datetime import datetime, timedelta
from dataclasses import dataclass, field
from typing import List, Dict, Optional, Tuple

import requests as _requests
import numpy as np
import pandas as pd

import plotly.graph_objects as go
from plotly.subplots import make_subplots
import plotly.io as pio

import ipywidgets as widgets
from IPython.display import display, clear_output, HTML

from tqdm.auto import tqdm

warnings.filterwarnings('ignore')

# yfinance 備援
try:
    import yfinance as yf
    _HAS_YF = True
except ImportError:
    _HAS_YF = False

# Colab 環境偵測
try:
    import google.colab
    _IN_COLAB = True
    pio.renderers.default = 'colab'
except ImportError:
    _IN_COLAB = False

# pandas 版本檢查
_PD_VER = tuple(int(x) for x in pd.__version__.split('.')[:2])
_MONTH_RULE = 'ME' if _PD_VER >= (2, 1) else 'M'

# ============================================================
# 全域設定 (依規格書)
# ============================================================

CACHE_DIR = Path('./cache')
CACHE_DIR.mkdir(parents=True, exist_ok=True)

CACHE_EXPIRY_HOURS = 12

# ZigZag 參數 (規格書 2.2)
ZIGZAG_THRESHOLD = 0.03    # 3% 回調門檻
ZIGZAG_MIN_BARS = 3        # 最小跨度

# 形態參數 (規格書 3.x)
TOLERANCE = 0.03           # 誤差容忍度 3%
NECKLINE_MIN_HEIGHT = 0.08 # W底/M頭頸線最小高度 8%
MIN_PATTERN_BARS = 15      # 形態最小跨度
HS_SYMMETRY_TOL = 0.10     # 頭肩對稱容差 10%
NECKLINE_MAX_SLOPE = 30    # 頸線最大斜率度數

# 預判觸發參數 (規格書 4.x)
TRIGGER_THRESHOLD = 0.02   # 2% 反彈/回落觸發

# 評分參數 (規格書 5.x)
QUALITY_THRESHOLD = 65     # 品質分數門檻
WEIGHT_VOLUME = 0.40       # 成交量權重 40%
WEIGHT_TREND = 0.30        # 趨勢位置權重 30%
WEIGHT_MORPHOLOGY = 0.30   # 形態完整度權重 30%

# 時框設定
TIMEFRAMES = ['1M', '1W', '1D']
TF_LABELS = {'1D': '日線', '1W': '週線', '1M': '月線'}

# API 速率控制
TIINGO_RATE_DELAY = 1.5
FINMIND_RATE_DELAY = 0.35

# 精選清單
QUICK_LIST = {
    'TW': [
        '2330','2454','2317','2382','2303','3711','2379','3034',
        '6770','2408','3231','2354','2395','3008','2474','6505',
        '2881','2882','2884','2886','2891','2892','5880','2885',
        '1301','1303','1326','2603','2609','2615','2002','1402',
        '6446','4904','3045','2912','9910','2207','1216','2105',
    ],
    'US': [
        'AAPL','MSFT','GOOGL','AMZN','META','NVDA','TSLA','NFLX',
        'AMD','INTC','AVGO','QCOM','MU','MRVL','AMAT','LRCX',
        'KLAC','ON','TXN','ADI','CRM','ORCL','ADBE','NOW',
        'SNOW','PLTR','PANW','CRWD','DDOG','NET','ZS','MNDY',
        'SHOP','PINS','SNAP','ROKU','PYPL','COIN','UBER',
        'LLY','UNH','JNJ','ABBV','MRK','PFE','TMO','ISRG',
        'JPM','BAC','GS','MS','V','MA','AXP','BLK',
        'XOM','CVX','COP','SLB','FCX','NEM',
        'WMT','COST','HD','NKE','SBUX','MCD','DIS','ABNB',
        'SPY','QQQ','IWM','XLF','XLE','XLK','SOXX','ARKK',
    ],
}

# ============================================================
# 數據結構 (規格書 6)
# ============================================================

@dataclass
class Pivot:
    """ZigZag 轉折點"""
    idx: int           # K線索引
    price: float       # 價格
    type: str          # 'PEAK' 或 'VALLEY'
    date: str          # 日期字串

@dataclass
class Pattern:
    """識別到的形態"""
    type: str          # DOUBLE_BOTTOM, DOUBLE_TOP, HS_BOTTOM, HS_TOP, CUP_HANDLE
    direction: str     # bullish 或 bearish
    pivots: List[Pivot]
    neckline: float
    end_idx: int

@dataclass
class PatternSignal:
    """輸出訊號 (規格書 6)"""
    symbol: str
    pattern_type: str
    signal_type: str   # PREDICTIVE_LONG 或 PREDICTIVE_SHORT
    timestamp: str
    timeframe: str
    quality_score: int
    entry_zone: float
    stop_loss: float
    target_neckline: float
    measured_move_target: float
    risk_reward_ratio: float
    pattern_details: Dict = field(default_factory=dict)
    score_breakdown: Dict = field(default_factory=dict)

# ============================================================
# 掃描控制
# ============================================================

class ScanControl:
    def __init__(self):
        self.stop_flag = False
    def stop(self):
        self.stop_flag = True
    def reset(self):
        self.stop_flag = False
    @property
    def should_stop(self):
        return self.stop_flag

scan_ctrl = ScanControl()

# ============================================================
# Ticker Registry
# ============================================================

class TickerRegistry:
    _cache_file = CACHE_DIR / '_ticker_registry.json'

    @staticmethod
    def _load():
        try:
            return json.loads(TickerRegistry._cache_file.read_text())
        except Exception:
            return {}

    @staticmethod
    def _save(d):
        TickerRegistry._cache_file.write_text(json.dumps(d, ensure_ascii=False))

    @staticmethod
    def fetch_us(tiingo_key: str) -> List[str]:
        cache = TickerRegistry._load()
        if cache.get('US') and cache.get('US_ts'):
            try:
                age = (datetime.now() - datetime.fromisoformat(cache['US_ts'])).total_seconds()
                if age < 86400:
                    return cache['US']
            except Exception:
                pass

        url = 'https://apimedia.tiingo.com/docs/tiingo/daily/supported_tickers.zip'
        r = _requests.get(url, timeout=60)
        if r.status_code != 200:
            raise RuntimeError(f'Tiingo 清單下載失敗: HTTP {r.status_code}')

        z = zipfile.ZipFile(io.BytesIO(r.content))
        with z.open(z.namelist()[0]) as f:
            rows = list(csv.DictReader(io.TextIOWrapper(f)))

        cutoff = (datetime.now() - timedelta(days=7)).strftime('%Y-%m-%d')
        tickers = sorted(set(
            row['ticker'] for row in rows
            if row.get('exchange') in ('NYSE', 'NASDAQ')
            and row.get('assetType') == 'Stock'
            and row.get('endDate', '') >= cutoff
        ))

        cache['US'] = tickers
        cache['US_ts'] = datetime.now().isoformat()
        TickerRegistry._save(cache)
        return tickers

    @staticmethod
    def fetch_tw(finmind_token: str = '') -> List[str]:
        cache = TickerRegistry._load()
        if cache.get('TW') and cache.get('TW_ts'):
            try:
                age = (datetime.now() - datetime.fromisoformat(cache['TW_ts'])).total_seconds()
                if age < 86400:
                    return cache['TW']
            except Exception:
                pass

        params = {'dataset': 'TaiwanStockInfo'}
        if finmind_token:
            params['token'] = finmind_token
        r = _requests.get('https://api.finmindtrade.com/api/v4/data', params=params, timeout=30)
        data = r.json()
        if data.get('status') != 200:
            raise RuntimeError(f'FinMind 錯誤: {data.get("msg")}')

        tickers = sorted(set(
            s['stock_id'] for s in data['data']
            if s.get('type') in ('twse', 'tpex')
        ))

        cache['TW'] = tickers
        cache['TW_ts'] = datetime.now().isoformat()
        TickerRegistry._save(cache)
        return tickers

# ============================================================
# DataManager (資料管理)
# ============================================================

class DataManager:
    def __init__(self, finmind_token='', tiingo_key=''):
        self.fm_token = finmind_token
        self.tg_key = tiingo_key
        self.session = _requests.Session()
        self._last_us = 0
        self._last_tw = 0
        self._backoff = 0
        self.stats = {'api_us': 0, 'api_tw': 0, 'yf_fallback': 0, 'cache': 0, 'err': 0}

    def _cache_path(self, ticker, market):
        return CACHE_DIR / f"{market}_{ticker.replace('.','_')}.csv"

    def _cache_ok(self, p):
        if not p.exists():
            return False
        age = (datetime.now() - datetime.fromtimestamp(p.stat().st_mtime)).total_seconds()
        return age < CACHE_EXPIRY_HOURS * 3600

    def _wait_us(self):
        delay = TIINGO_RATE_DELAY + self._backoff
        gap = time.time() - self._last_us
        if gap < delay:
            time.sleep(delay - gap)
        self._last_us = time.time()

    def _wait_tw(self):
        gap = time.time() - self._last_tw
        if gap < FINMIND_RATE_DELAY:
            time.sleep(FINMIND_RATE_DELAY - gap)
        self._last_tw = time.time()

    def _fetch_us(self, ticker):
        self._wait_us()
        end = datetime.now()
        start = end - timedelta(days=2000)
        url = f'https://api.tiingo.com/tiingo/daily/{ticker}/prices'
        params = {
            'startDate': start.strftime('%Y-%m-%d'),
            'endDate': end.strftime('%Y-%m-%d'),
            'token': self.tg_key
        }
        r = self.session.get(url, params=params, timeout=15)
        if r.status_code == 429:
            self._backoff = min(30, self._backoff + 3)
            time.sleep(self._backoff)
            self._last_us = time.time()
            r = self.session.get(url, params=params, timeout=15)
        if r.status_code != 200:
            raise ValueError(f'Tiingo HTTP {r.status_code}')
        data = r.json()
        if not data:
            raise ValueError('Tiingo 無數據')
        df = pd.DataFrame(data)
        df['date'] = pd.to_datetime(df['date']).dt.tz_localize(None)
        df = df.set_index('date')
        adj_cols = ['adjOpen', 'adjHigh', 'adjLow', 'adjClose', 'adjVolume']
        df = df[adj_cols].copy()
        df.columns = ['Open', 'High', 'Low', 'Close', 'Volume']
        self.stats['api_us'] += 1
        self._backoff = max(0, self._backoff - 0.5)
        return df

    def _fetch_us_yf(self, ticker):
        if not _HAS_YF:
            raise ImportError('yfinance 未安裝')
        end = datetime.now()
        start = end - timedelta(days=2000)
        df = yf.download(ticker, start=start.strftime('%Y-%m-%d'),
                         end=end.strftime('%Y-%m-%d'), progress=False, auto_adjust=True)
        if df is None or df.empty:
            raise ValueError('yfinance 無數據')
        if isinstance(df.columns, pd.MultiIndex):
            df.columns = df.columns.get_level_values(0)
        self.stats['yf_fallback'] += 1
        return df

    def _fetch_tw(self, ticker):
        self._wait_tw()
        end = datetime.now()
        start = end - timedelta(days=2000)
        params = {
            'dataset': 'TaiwanStockPrice',
            'data_id': ticker,
            'start_date': start.strftime('%Y-%m-%d'),
            'end_date': end.strftime('%Y-%m-%d'),
        }
        if self.fm_token:
            params['token'] = self.fm_token
        r = self.session.get('https://api.finmindtrade.com/api/v4/data', params=params, timeout=15)
        data = r.json()
        if data.get('status') != 200:
            raise ValueError(f'FinMind: {data.get("msg","")}')
        rows = data['data']
        if not rows:
            raise ValueError('FinMind 無數據')
        df = pd.DataFrame(rows)
        df['date'] = pd.to_datetime(df['date'])
        df = df.set_index('date')
        self.stats['api_tw'] += 1
        return df

    def _standardize(self, df):
        col_map = {
            'open': 'Open', 'high': 'High', 'low': 'Low',
            'close': 'Close', 'volume': 'Volume',
            'max': 'High', 'min': 'Low', 'Trading_Volume': 'Volume',
        }
        df = df.rename(columns=col_map)
        for target in ['Open', 'High', 'Low', 'Close', 'Volume']:
            if target not in df.columns:
                for c in df.columns:
                    if c.lower() == target.lower():
                        df = df.rename(columns={c: target})
                        break
        keep = [c for c in ['Open','High','Low','Close','Volume'] if c in df.columns]
        df = df[keep].copy()
        if hasattr(df.index, 'tz') and df.index.tz is not None:
            df.index = df.index.tz_localize(None)
        for c in df.columns:
            df[c] = pd.to_numeric(df[c], errors='coerce')
        return df.sort_index().dropna()

    @staticmethod
    def resample(df, timeframe='1D'):
        if timeframe == '1D' or df is None or df.empty:
            return df
        rule = {'1W': 'W', '1M': _MONTH_RULE}[timeframe]
        agg = {'Open': 'first', 'High': 'max', 'Low': 'min', 'Close': 'last', 'Volume': 'sum'}
        return df.resample(rule).agg(agg).dropna()

    def get(self, ticker, market='TW'):
        cp = self._cache_path(ticker, market)
        try:
            if self._cache_ok(cp):
                df = pd.read_csv(cp, index_col=0, parse_dates=True)
                self.stats['cache'] += 1
                return self._standardize(df)
            if market.upper() == 'TW':
                df = self._fetch_tw(ticker)
            else:
                try:
                    df = self._fetch_us(ticker)
                except Exception:
                    df = self._fetch_us_yf(ticker)
            df = self._standardize(df)
            df.to_csv(cp)
            return df
        except Exception:
            self.stats['err'] += 1
            return None

# ============================================================
# 數據前處理 (規格書 2.1)
# ============================================================

def preprocess(df: pd.DataFrame) -> pd.DataFrame:
    """
    清洗規則：
    1. 剔除成交量為 0 的非交易日
    2. 剔除 High == Low (無波動) 的死魚盤 K 線
    """
    if df is None or df.empty:
        return df
    df = df[df['Volume'] > 0].copy()
    df = df[df['High'] != df['Low']].copy()
    return df

# ============================================================
# ZigZag 演算法 (規格書 2.2)
# ============================================================

def zigzag(df: pd.DataFrame, threshold: float = ZIGZAG_THRESHOLD, 
           min_bars: int = ZIGZAG_MIN_BARS) -> List[Pivot]:
    """
    ZigZag 波段識別演算法
    - 僅當價格自極值回調超過 threshold 時，確認該極值點
    - 輸出: Pivot 序列 [P0, P1, P2, ...] (P0 為最新轉折點)
    """
    if df is None or len(df) < min_bars * 2:
        return []
    
    highs = df['High'].values
    lows = df['Low'].values
    dates = df.index
    n = len(df)
    
    pivots = []
    
    # 初始化：找第一個極值
    if highs[0] >= lows[0]:
        last_type = 'PEAK'
        last_idx = 0
        last_price = highs[0]
    else:
        last_type = 'VALLEY'
        last_idx = 0
        last_price = lows[0]
    
    # 追蹤當前潛在極值
    potential_high_idx = 0
    potential_high = highs[0]
    potential_low_idx = 0
    potential_low = lows[0]
    
    for i in range(1, n):
        # 更新潛在高點
        if highs[i] > potential_high:
            potential_high = highs[i]
            potential_high_idx = i
        
        # 更新潛在低點
        if lows[i] < potential_low:
            potential_low = lows[i]
            potential_low_idx = i
        
        # 檢查是否確認轉折
        if last_type == 'PEAK':
            # 當前趨勢向下，尋找 VALLEY
            # 如果從潛在低點反彈超過 threshold，確認該低點
            if potential_high > 0 and (highs[i] - potential_low) / potential_low >= threshold:
                if i - potential_low_idx >= min_bars:
                    date_str = str(dates[potential_low_idx].date()) if hasattr(dates[potential_low_idx], 'date') else str(dates[potential_low_idx])
                    pivots.append(Pivot(potential_low_idx, potential_low, 'VALLEY', date_str))
                    last_type = 'VALLEY'
                    last_idx = potential_low_idx
                    last_price = potential_low
                    # 重置潛在高點
                    potential_high = highs[i]
                    potential_high_idx = i
        else:
            # 當前趨勢向上，尋找 PEAK
            # 如果從潛在高點回落超過 threshold，確認該高點
            if potential_low > 0 and (potential_high - lows[i]) / potential_high >= threshold:
                if i - potential_high_idx >= min_bars:
                    date_str = str(dates[potential_high_idx].date()) if hasattr(dates[potential_high_idx], 'date') else str(dates[potential_high_idx])
                    pivots.append(Pivot(potential_high_idx, potential_high, 'PEAK', date_str))
                    last_type = 'PEAK'
                    last_idx = potential_high_idx
                    last_price = potential_high
                    # 重置潛在低點
                    potential_low = lows[i]
                    potential_low_idx = i
    
    # 逆序返回 (P0 為最新)
    return list(reversed(pivots))

# ============================================================
# 形態識別 (規格書 3.x)
# ============================================================

class PatternDetector:
    """形態識別器"""
    
    @staticmethod
    def detect_double_bottom(pivots: List[Pivot], df: pd.DataFrame) -> Optional[Pattern]:
        """
        雙重底 (W-Bottom) - 4 點識別
        條件：
        1. P4 (左腳), P2 (右腳) 為 VALLEY
        2. 雙腳水平: |P4 - P2| / P4 <= 3%
        3. P3 (頸線) 比 P4 高出至少 8%
        4. P4 與 P2 間隔至少 15 個交易單位
        """
        if len(pivots) < 4:
            return None
        
        # 取最近 4 個轉折點 (P0 最新)
        p0, p1, p2, p3 = pivots[0], pivots[1], pivots[2], pivots[3]
        
        # 檢查 P2, P4 都是 VALLEY (P4=p3, P2=p1 in 0-indexed)
        # 實際上 4 點是 P1, P2, P3, P4，P4 最舊
        # pivots[0]=P1(最新), pivots[1]=P2, pivots[2]=P3, pivots[3]=P4(最舊)
        
        # W底: P4(VALLEY) - P3(PEAK) - P2(VALLEY) - P1(PEAK/當前)
        # 需要 P4 和 P2 是 VALLEY，P3 是 PEAK
        
        if p3.type != 'VALLEY' or p1.type != 'VALLEY':
            return None
        if p2.type != 'PEAK':
            return None
        
        left_bottom = p3.price   # P4 (左腳)
        right_bottom = p1.price  # P2 (右腳)
        neckline = p2.price      # P3 (頸線高點)
        
        # 條件 2: 雙腳水平 3% 內
        if abs(left_bottom - right_bottom) / left_bottom > TOLERANCE:
            return None
        
        # 條件 3: 頸線高度 8%+
        avg_bottom = (left_bottom + right_bottom) / 2
        if (neckline - avg_bottom) / avg_bottom < NECKLINE_MIN_HEIGHT:
            return None
        
        # 條件 4: 時間跨度 15+
        if p1.idx - p3.idx < MIN_PATTERN_BARS:
            return None
        
        return Pattern(
            type='DOUBLE_BOTTOM',
            direction='bullish',
            pivots=[p3, p2, p1, p0] if p0 else [p3, p2, p1],
            neckline=neckline,
            end_idx=p1.idx
        )
    
    @staticmethod
    def detect_double_top(pivots: List[Pivot], df: pd.DataFrame) -> Optional[Pattern]:
        """
        雙重頂 (M-Top) - 4 點識別
        """
        if len(pivots) < 4:
            return None
        
        p0, p1, p2, p3 = pivots[0], pivots[1], pivots[2], pivots[3]
        
        # M頭: P4(PEAK) - P3(VALLEY) - P2(PEAK) - P1(VALLEY/當前)
        if p3.type != 'PEAK' or p1.type != 'PEAK':
            return None
        if p2.type != 'VALLEY':
            return None
        
        left_top = p3.price
        right_top = p1.price
        neckline = p2.price
        
        # 雙頭水平 3% 內
        if abs(left_top - right_top) / left_top > TOLERANCE:
            return None
        
        # 頸線深度 8%+
        avg_top = (left_top + right_top) / 2
        if (avg_top - neckline) / avg_top < NECKLINE_MIN_HEIGHT:
            return None
        
        # 時間跨度 15+
        if p1.idx - p3.idx < MIN_PATTERN_BARS:
            return None
        
        return Pattern(
            type='DOUBLE_TOP',
            direction='bearish',
            pivots=[p3, p2, p1, p0] if p0 else [p3, p2, p1],
            neckline=neckline,
            end_idx=p1.idx
        )
    
    @staticmethod
    def detect_hs_bottom(pivots: List[Pivot], df: pd.DataFrame) -> Optional[Pattern]:
        """
        頭肩底 (H&S Bottom) - 5 點識別
        條件：
        1. P5(左肩底), P3(頭底), P1(右肩底) 為 VALLEY
        2. P3 < P5 且 P3 < P1 (頭部最低)
        3. |P5 - P1| / P3 < 10%
        4. 右肩形成時間 > 左肩時間 × 0.3
        5. 頸線斜率 ≤ 30°
        """
        if len(pivots) < 5:
            return None
        
        p0, p1, p2, p3, p4 = pivots[0], pivots[1], pivots[2], pivots[3], pivots[4]
        
        # 檢查極值屬性：P5, P3, P1 是 VALLEY
        # p4=P5, p2=P3(頭), p0=P1(如果有的話)或p1=P1
        # 實際: pivots[4]=左肩, pivots[2]=頭, pivots[0]=右肩 (如果都是VALLEY)
        
        # 頭肩底: VALLEY(左肩) - PEAK - VALLEY(頭) - PEAK - VALLEY(右肩)
        if p4.type != 'VALLEY' or p2.type != 'VALLEY' or p0.type != 'VALLEY':
            return None
        if p3.type != 'PEAK' or p1.type != 'PEAK':
            return None
        
        left_shoulder = p4.price
        head = p2.price
        right_shoulder = p0.price
        
        # 條件 2: 頭部最低
        if not (head < left_shoulder and head < right_shoulder):
            return None
        
        # 條件 3: 對稱性 10%
        if abs(left_shoulder - right_shoulder) / head > HS_SYMMETRY_TOL:
            return None
        
        # 條件 4: 時間對稱
        left_time = p2.idx - p4.idx
        right_time = p0.idx - p2.idx
        if right_time < left_time * 0.3:
            return None
        
        # 條件 5: 頸線斜率 (連接 P4 與 P2 的高點)
        neckline_left = p3.price   # 左頸線點
        neckline_right = p1.price  # 右頸線點
        time_diff = p1.idx - p3.idx
        if time_diff > 0:
            price_diff = abs(neckline_right - neckline_left)
            avg_price = (neckline_left + neckline_right) / 2
            slope_ratio = price_diff / avg_price / time_diff * 100
            slope_deg = math.degrees(math.atan(slope_ratio))
            if abs(slope_deg) > NECKLINE_MAX_SLOPE:
                return None
        
        neckline = (neckline_left + neckline_right) / 2
        
        return Pattern(
            type='HS_BOTTOM',
            direction='bullish',
            pivots=[p4, p3, p2, p1, p0],
            neckline=neckline,
            end_idx=p0.idx
        )
    
    @staticmethod
    def detect_hs_top(pivots: List[Pivot], df: pd.DataFrame) -> Optional[Pattern]:
        """
        頭肩頂 (H&S Top) - 5 點識別
        """
        if len(pivots) < 5:
            return None
        
        p0, p1, p2, p3, p4 = pivots[0], pivots[1], pivots[2], pivots[3], pivots[4]
        
        # 頭肩頂: PEAK(左肩) - VALLEY - PEAK(頭) - VALLEY - PEAK(右肩)
        if p4.type != 'PEAK' or p2.type != 'PEAK' or p0.type != 'PEAK':
            return None
        if p3.type != 'VALLEY' or p1.type != 'VALLEY':
            return None
        
        left_shoulder = p4.price
        head = p2.price
        right_shoulder = p0.price
        
        # 頭部最高
        if not (head > left_shoulder and head > right_shoulder):
            return None
        
        # 對稱性 10%
        if abs(left_shoulder - right_shoulder) / head > HS_SYMMETRY_TOL:
            return None
        
        # 時間對稱
        left_time = p2.idx - p4.idx
        right_time = p0.idx - p2.idx
        if right_time < left_time * 0.3:
            return None
        
        # 頸線斜率
        neckline_left = p3.price
        neckline_right = p1.price
        time_diff = p1.idx - p3.idx
        if time_diff > 0:
            price_diff = abs(neckline_right - neckline_left)
            avg_price = (neckline_left + neckline_right) / 2
            slope_ratio = price_diff / avg_price / time_diff * 100
            slope_deg = math.degrees(math.atan(slope_ratio))
            if abs(slope_deg) > NECKLINE_MAX_SLOPE:
                return None
        
        neckline = (neckline_left + neckline_right) / 2
        
        return Pattern(
            type='HS_TOP',
            direction='bearish',
            pivots=[p4, p3, p2, p1, p0],
            neckline=neckline,
            end_idx=p0.idx
        )
    
    @staticmethod
    def detect_cup_handle(df: pd.DataFrame, pivots: List[Pivot]) -> Optional[Pattern]:
        """
        杯柄形態 (Cup and Handle)
        條件：
        1. 杯身: 60-150 根 K 線內的 U 型結構
        2. 杯底位於區間中間 50% 時間帶內
        3. 柄部在 High_Right 之後
        4. 柄部回調 ≤ 杯身深度 × 0.382
        """
        if df is None or len(df) < 60:
            return None
        
        # 檢查最近 60-150 根 K 線
        lookback = min(150, len(df))
        recent = df.iloc[-lookback:]
        
        # 找杯身高點和低點
        high_left_idx = recent['High'].iloc[:lookback//3].idxmax()
        high_right_idx = recent['High'].iloc[-lookback//3:].idxmax()
        
        left_idx = recent.index.get_loc(high_left_idx)
        right_idx = recent.index.get_loc(high_right_idx)
        
        if right_idx - left_idx < 30:
            return None
        
        cup_section = recent.iloc[left_idx:right_idx+1]
        if len(cup_section) < 30:
            return None
        
        low_idx = cup_section['Low'].idxmin()
        low_pos = cup_section.index.get_loc(low_idx)
        
        # 杯底必須在中間 50%
        section_len = len(cup_section)
        if not (section_len * 0.25 < low_pos < section_len * 0.75):
            return None
        
        high_left = recent['High'].loc[high_left_idx]
        high_right = recent['High'].loc[high_right_idx]
        low_min = cup_section['Low'].loc[low_idx]
        
        cup_depth = ((high_left + high_right) / 2) - low_min
        
        # 檢查柄部 (杯身右側之後)
        if right_idx + 5 >= len(recent):
            return None
        
        handle_section = recent.iloc[right_idx:]
        if len(handle_section) < 3:
            return None
        
        handle_low = handle_section['Low'].min()
        handle_pullback = high_right - handle_low
        
        # 柄部回調 ≤ 杯身 × 0.382
        if handle_pullback > cup_depth * 0.382:
            return None
        
        # 當前價格
        current_close = df['Close'].iloc[-1]
        
        # 創建 Pattern
        neckline = (high_left + high_right) / 2
        
        # 構造虛擬 Pivots
        date_low = str(low_idx.date()) if hasattr(low_idx, 'date') else str(low_idx)
        cup_low_pivot = Pivot(len(df) - lookback + low_pos, low_min, 'VALLEY', date_low)
        
        return Pattern(
            type='CUP_HANDLE',
            direction='bullish',
            pivots=[cup_low_pivot],
            neckline=neckline,
            end_idx=len(df) - 1
        )
    
    @staticmethod
    def detect_all(df: pd.DataFrame, pivots: List[Pivot]) -> List[Pattern]:
        """檢測所有形態"""
        patterns = []
        
        # 雙重底
        pat = PatternDetector.detect_double_bottom(pivots, df)
        if pat:
            patterns.append(pat)
        
        # 雙重頂
        pat = PatternDetector.detect_double_top(pivots, df)
        if pat:
            patterns.append(pat)
        
        # 頭肩底
        pat = PatternDetector.detect_hs_bottom(pivots, df)
        if pat:
            patterns.append(pat)
        
        # 頭肩頂
        pat = PatternDetector.detect_hs_top(pivots, df)
        if pat:
            patterns.append(pat)
        
        # 杯柄
        pat = PatternDetector.detect_cup_handle(df, pivots)
        if pat:
            patterns.append(pat)
        
        return patterns

# ============================================================
# 預判型觸發機制 (規格書 4.x)
# ============================================================

class PredictiveTrigger:
    """預判型觸發檢查"""
    
    @staticmethod
    def check_trigger(df: pd.DataFrame, pattern: Pattern) -> bool:
        """檢查是否觸發預判訊號"""
        if df is None or df.empty:
            return False
        
        current_close = df['Close'].iloc[-1]
        
        if pattern.direction == 'bullish':
            # 看多: Close > Low_Right × 1.02
            right_low = PredictiveTrigger._get_right_low(pattern)
            if right_low and current_close > right_low * (1 + TRIGGER_THRESHOLD):
                return True
        else:
            # 看空: Close < High_Right × 0.98
            right_high = PredictiveTrigger._get_right_high(pattern)
            if right_high and current_close < right_high * (1 - TRIGGER_THRESHOLD):
                return True
        
        return False
    
    @staticmethod
    def check_invalidation(df: pd.DataFrame, pattern: Pattern) -> bool:
        """檢查形態是否失效"""
        if df is None or df.empty:
            return False
        
        current_close = df['Close'].iloc[-1]
        
        if pattern.direction == 'bullish':
            right_low = PredictiveTrigger._get_right_low(pattern)
            if right_low and current_close < right_low:
                return True
        else:
            right_high = PredictiveTrigger._get_right_high(pattern)
            if right_high and current_close > right_high:
                return True
        
        return False
    
    @staticmethod
    def _get_right_low(pattern: Pattern) -> Optional[float]:
        """取得右側最低點"""
        if pattern.type == 'DOUBLE_BOTTOM':
            # P2 (右腳) 是 pivots[2] 或看最新的 VALLEY
            for p in pattern.pivots:
                if p.type == 'VALLEY':
                    return p.price
        elif pattern.type == 'HS_BOTTOM':
            # 右肩底
            if pattern.pivots and pattern.pivots[-1].type == 'VALLEY':
                return pattern.pivots[-1].price
        elif pattern.type == 'CUP_HANDLE':
            if pattern.pivots:
                return pattern.pivots[0].price
        return None
    
    @staticmethod
    def _get_right_high(pattern: Pattern) -> Optional[float]:
        """取得右側最高點"""
        if pattern.type == 'DOUBLE_TOP':
            for p in pattern.pivots:
                if p.type == 'PEAK':
                    return p.price
        elif pattern.type == 'HS_TOP':
            if pattern.pivots and pattern.pivots[-1].type == 'PEAK':
                return pattern.pivots[-1].price
        return None

# ============================================================
# 評分系統 (規格書 5.x)
# ============================================================

class QualityScorer:
    """品質評分器"""
    
    @staticmethod
    def score(df: pd.DataFrame, pattern: Pattern) -> Tuple[int, Dict]:
        """
        計算品質分數
        - 成交量 (40%): 右側量縮 +20, 反彈出量 +20
        - 趨勢位置 (30%): BB下緣/200MA附近 +30
        - 形態完整度 (30%): 對稱性 +15, 水平度 +15
        """
        breakdown = {'volume': 0, 'trend': 0, 'morphology': 0}
        
        # 成交量評分
        vol_score = QualityScorer._score_volume(df, pattern)
        breakdown['volume'] = vol_score
        
        # 趨勢位置評分
        trend_score = QualityScorer._score_trend(df, pattern)
        breakdown['trend'] = trend_score
        
        # 形態完整度評分
        morph_score = QualityScorer._score_morphology(pattern)
        breakdown['morphology'] = morph_score
        
        # 加權總分
        total = (
            vol_score * WEIGHT_VOLUME +
            trend_score * WEIGHT_TREND +
            morph_score * WEIGHT_MORPHOLOGY
        )
        
        return int(round(total)), breakdown
    
    @staticmethod
    def _score_volume(df: pd.DataFrame, pattern: Pattern) -> int:
        """成交量評分 (滿分 100)"""
        score = 0
        
        if len(df) < 40:
            return 50  # 數據不足，給中間分
        
        # 計算 20 日均量
        vol_ma20 = df['Volume'].rolling(20).mean()
        
        end_idx = pattern.end_idx
        if end_idx < 20 or end_idx >= len(df):
            return 50
        
        # 右側量縮檢查
        if pattern.type in ['DOUBLE_BOTTOM', 'HS_BOTTOM', 'CUP_HANDLE']:
            # 找左右腳的平均成交量
            mid_idx = end_idx - 10
            if mid_idx > 10:
                left_vol = df['Volume'].iloc[mid_idx-10:mid_idx].mean()
                right_vol = df['Volume'].iloc[mid_idx:end_idx].mean()
                if right_vol < left_vol:
                    score += 50  # 右側量縮 +50 (換算成 +20 權重)
        
        # 反彈出量檢查
        today_vol = df['Volume'].iloc[-1]
        avg_vol = vol_ma20.iloc[-1] if not pd.isna(vol_ma20.iloc[-1]) else df['Volume'].mean()
        if today_vol > avg_vol:
            score += 50  # 反彈出量 +50
        
        return min(100, score)
    
    @staticmethod
    def _score_trend(df: pd.DataFrame, pattern: Pattern) -> int:
        """趨勢位置評分 (滿分 100)"""
        if len(df) < 200:
            return 50
        
        current_close = df['Close'].iloc[-1]
        
        # 計算 200MA
        ma200 = df['Close'].rolling(200).mean().iloc[-1]
        
        # 計算 Bollinger Band
        ma20 = df['Close'].rolling(20).mean().iloc[-1]
        std20 = df['Close'].rolling(20).std().iloc[-1]
        bb_lower = ma20 - 2 * std20
        bb_upper = ma20 + 2 * std20
        
        score = 0
        
        if pattern.direction == 'bullish':
            # 多頭形態：接近 BB 下緣或 200MA
            if current_close <= bb_lower * 1.02:
                score = 100
            elif current_close <= ma200 * 1.05:
                score = 80
            elif current_close <= ma20:
                score = 50
        else:
            # 空頭形態：接近 BB 上緣
            if current_close >= bb_upper * 0.98:
                score = 100
            elif current_close >= ma200 * 0.95:
                score = 80
            elif current_close >= ma20:
                score = 50
        
        return score
    
    @staticmethod
    def _score_morphology(pattern: Pattern) -> int:
        """形態完整度評分 (滿分 100)"""
        score = 0
        
        if not pattern.pivots or len(pattern.pivots) < 2:
            return 50
        
        # 對稱性評分 (時間對稱)
        if pattern.type in ['DOUBLE_BOTTOM', 'DOUBLE_TOP'] and len(pattern.pivots) >= 3:
            p_left = pattern.pivots[0]
            p_mid = pattern.pivots[1]
            p_right = pattern.pivots[2]
            left_time = p_mid.idx - p_left.idx
            right_time = p_right.idx - p_mid.idx
            if left_time > 0:
                time_ratio = abs(left_time - right_time) / left_time
                if time_ratio < 0.2:
                    score += 50  # 對稱性好
                elif time_ratio < 0.4:
                    score += 25
        
        # 水平度評分 (價格水平)
        if pattern.type == 'DOUBLE_BOTTOM' and len(pattern.pivots) >= 3:
            left_price = pattern.pivots[0].price
            right_price = pattern.pivots[2].price if len(pattern.pivots) > 2 else pattern.pivots[1].price
            price_diff = abs(left_price - right_price) / left_price
            if price_diff < 0.01:
                score += 50  # 非常水平
            elif price_diff < 0.02:
                score += 35
            elif price_diff < 0.03:
                score += 20
        elif pattern.type == 'DOUBLE_TOP' and len(pattern.pivots) >= 3:
            left_price = pattern.pivots[0].price
            right_price = pattern.pivots[2].price if len(pattern.pivots) > 2 else pattern.pivots[1].price
            price_diff = abs(left_price - right_price) / left_price
            if price_diff < 0.01:
                score += 50
            elif price_diff < 0.02:
                score += 35
            elif price_diff < 0.03:
                score += 20
        else:
            score += 30  # 其他形態給基本分
        
        return min(100, score)

# ============================================================
# 目標價計算 (規格書 6)
# ============================================================

class TargetCalculator:
    """目標價計算器"""
    
    @staticmethod
    def calculate(df: pd.DataFrame, pattern: Pattern) -> Dict:
        """計算停損和目標價"""
        result = {
            'entry_zone': round(df['Close'].iloc[-1], 2),
            'stop_loss': 0.0,
            'target_neckline': round(pattern.neckline, 2),
            'measured_move_target': 0.0
        }
        
        if pattern.type == 'DOUBLE_BOTTOM':
            # 停損: 右腳低點
            right_low = None
            for p in pattern.pivots:
                if p.type == 'VALLEY':
                    right_low = p.price
                    break
            if right_low:
                result['stop_loss'] = round(right_low * 0.99, 2)  # 再下 1%
            
            # 等幅測距: neckline + (neckline - avg_bottom)
            avg_bottom = sum(p.price for p in pattern.pivots if p.type == 'VALLEY') / max(1, sum(1 for p in pattern.pivots if p.type == 'VALLEY'))
            depth = pattern.neckline - avg_bottom
            result['measured_move_target'] = round(pattern.neckline + depth, 2)
        
        elif pattern.type == 'DOUBLE_TOP':
            # 停損: 右頭高點
            right_high = None
            for p in pattern.pivots:
                if p.type == 'PEAK':
                    right_high = p.price
                    break
            if right_high:
                result['stop_loss'] = round(right_high * 1.01, 2)
            
            # 等幅測距
            avg_top = sum(p.price for p in pattern.pivots if p.type == 'PEAK') / max(1, sum(1 for p in pattern.pivots if p.type == 'PEAK'))
            depth = avg_top - pattern.neckline
            result['measured_move_target'] = round(pattern.neckline - depth, 2)
        
        elif pattern.type == 'HS_BOTTOM':
            # 停損: 頭部低點
            head_low = min(p.price for p in pattern.pivots if p.type == 'VALLEY')
            result['stop_loss'] = round(head_low * 0.99, 2)
            
            # 等幅測距
            depth = pattern.neckline - head_low
            result['measured_move_target'] = round(pattern.neckline + depth, 2)
        
        elif pattern.type == 'HS_TOP':
            head_high = max(p.price for p in pattern.pivots if p.type == 'PEAK')
            result['stop_loss'] = round(head_high * 1.01, 2)
            
            depth = head_high - pattern.neckline
            result['measured_move_target'] = round(pattern.neckline - depth, 2)
        
        elif pattern.type == 'CUP_HANDLE':
            # 停損: 柄部低點
            if pattern.pivots:
                result['stop_loss'] = round(pattern.pivots[0].price * 0.99, 2)
            
            # 等幅測距
            cup_depth = pattern.neckline - pattern.pivots[0].price if pattern.pivots else 0
            result['measured_move_target'] = round(pattern.neckline + cup_depth, 2)
        
        return result

# ============================================================
# 市場掃描器
# ============================================================

class MarketScanner:
    def __init__(self, dm: DataManager, ctrl: ScanControl = None):
        self.dm = dm
        self.ctrl = ctrl or ScanControl()
    
    def scan(self, market: str, tickers: List[str]) -> List[PatternSignal]:
        results = []
        ok, fail = 0, 0
        n = len(tickers)
        
        print(f'\n{"="*55}')
        print(f'掃描 {market} ({n} 檔 x {len(TIMEFRAMES)} 時框)')
        print(f'{"="*55}')
        
        pbar = tqdm(tickers, desc=f'{market}', unit='檔')
        for ticker in pbar:
            if self.ctrl.should_stop:
                print(f'\n使用者中止 (已完成 {ok+fail}/{n})')
                break
            
            try:
                df_daily = self.dm.get(ticker, market)
                if df_daily is None or len(df_daily) < 60:
                    fail += 1
                    continue
                
                # 前處理
                df_daily = preprocess(df_daily)
                if df_daily is None or len(df_daily) < 60:
                    fail += 1
                    continue
                
                for tf in TIMEFRAMES:
                    df = DataManager.resample(df_daily, tf)
                    if df is None or len(df) < 60:
                        continue
                    
                    # 1. ZigZag 波段識別
                    pivots = zigzag(df)
                    if len(pivots) < 4:
                        continue
                    
                    # 2. 形態識別
                    patterns = PatternDetector.detect_all(df, pivots)
                    
                    for pat in patterns:
                        # 3. 預判觸發檢查
                        if not PredictiveTrigger.check_trigger(df, pat):
                            continue
                        
                        # 檢查是否失效
                        if PredictiveTrigger.check_invalidation(df, pat):
                            continue
                        
                        # 4. 評分
                        score, breakdown = QualityScorer.score(df, pat)
                        if score < QUALITY_THRESHOLD:
                            continue
                        
                        # 5. 計算目標價
                        targets = TargetCalculator.calculate(df, pat)
                        
                        # 計算盈虧比
                        entry = targets['entry_zone']
                        stop = targets['stop_loss']
                        target1 = targets['target_neckline']
                        
                        if pat.direction == 'bullish' and entry > stop:
                            risk = entry - stop
                            reward = target1 - entry
                            rr_ratio = round(reward / risk, 2) if risk > 0 else 0
                        elif pat.direction == 'bearish' and stop > entry:
                            risk = stop - entry
                            reward = entry - target1
                            rr_ratio = round(reward / risk, 2) if risk > 0 else 0
                        else:
                            rr_ratio = 0
                        
                        # 6. 組裝輸出
                        signal = PatternSignal(
                            symbol=f"{ticker}.{market}",
                            pattern_type=pat.type,
                            signal_type='PREDICTIVE_LONG' if pat.direction == 'bullish' else 'PREDICTIVE_SHORT',
                            timestamp=str(df.index[-1].date()) if hasattr(df.index[-1], 'date') else str(df.index[-1]),
                            timeframe=tf,
                            quality_score=score,
                            entry_zone=targets['entry_zone'],
                            stop_loss=targets['stop_loss'],
                            target_neckline=targets['target_neckline'],
                            measured_move_target=targets['measured_move_target'],
                            risk_reward_ratio=rr_ratio,
                            pattern_details={
                                'pivots': [(p.date, p.price, p.type) for p in pat.pivots],
                                'neckline': pat.neckline,
                            },
                            score_breakdown=breakdown
                        )
                        results.append(signal)
                
                ok += 1
            except Exception as e:
                fail += 1
            
            pbar.set_postfix({
                'OK': ok, 'Fail': fail,
                '訊號': len(results),
                '快取': self.dm.stats['cache']
            })
        
        s = self.dm.stats
        yf_info = f' yf={s["yf_fallback"]}' if s['yf_fallback'] else ''
        print(f'完成 {ok}/{ok+fail} | API: US={s["api_us"]} TW={s["api_tw"]}{yf_info} | 快取={s["cache"]} | 錯誤={s["err"]}')
        print(f'找到 {len(results)} 個預判訊號 (品質>={QUALITY_THRESHOLD})')
        return results
    
    @staticmethod
    def to_df(results: List[PatternSignal]) -> pd.DataFrame:
        if not results:
            return pd.DataFrame(columns=['代號','市場','週期','形態','訊號','分數','進場','停損','目標1','目標2','R:R'])
        
        rows = []
        for r in results:
            symbol_parts = r.symbol.split('.')
            ticker = symbol_parts[0]
            market = symbol_parts[1] if len(symbol_parts) > 1 else ''
            
            pattern_names = {
                'DOUBLE_BOTTOM': 'W底',
                'DOUBLE_TOP': 'M頭',
                'HS_BOTTOM': '頭肩底',
                'HS_TOP': '頭肩頂',
                'CUP_HANDLE': '杯柄'
            }
            
            rows.append({
                '代號': ticker,
                '市場': market,
                '週期': TF_LABELS.get(r.timeframe, r.timeframe),
                '形態': pattern_names.get(r.pattern_type, r.pattern_type),
                '訊號': '做多' if 'LONG' in r.signal_type else '做空',
                '分數': r.quality_score,
                '進場': r.entry_zone,
                '停損': r.stop_loss,
                '目標1': r.target_neckline,
                '目標2': r.measured_move_target,
                'R:R': f"1:{r.risk_reward_ratio}"
            })
        
        return pd.DataFrame(rows).sort_values('分數', ascending=False).reset_index(drop=True)

# ============================================================
# 繪圖
# ============================================================

def plot_stock(dm: DataManager, ticker: str, market: str, signal: PatternSignal = None):
    """繪製 K 線圖和形態標記"""
    df = dm.get(ticker, market)
    if df is None:
        print(f'無法取得 {ticker}')
        return
    
    df = preprocess(df)
    
    timeframe = '1D'
    if signal:
        timeframe = signal.timeframe
    df = DataManager.resample(df, timeframe)
    tf_label = TF_LABELS.get(timeframe, timeframe)
    
    fig = make_subplots(rows=2, cols=1, shared_xaxes=True,
                        vertical_spacing=0.03, row_heights=[0.7, 0.3])
    
    # K 線
    fig.add_trace(go.Candlestick(
        x=df.index, open=df['Open'], high=df['High'],
        low=df['Low'], close=df['Close'], name='K線',
        increasing_line_color='red', decreasing_line_color='green'
    ), row=1, col=1)
    
    # 成交量
    colors = ['red' if c >= o else 'green' for c, o in zip(df['Close'], df['Open'])]
    fig.add_trace(go.Bar(
        x=df.index, y=df['Volume'], marker_color=colors,
        name='成交量', opacity=0.6
    ), row=2, col=1)
    
    # 標記關鍵價位
    if signal:
        # 進場價
        fig.add_hline(y=signal.entry_zone, line_dash="solid", line_color="blue",
                      annotation_text=f"進場 {signal.entry_zone}", row=1, col=1)
        # 停損
        fig.add_hline(y=signal.stop_loss, line_dash="dash", line_color="red",
                      annotation_text=f"停損 {signal.stop_loss}", row=1, col=1)
        # 目標1
        fig.add_hline(y=signal.target_neckline, line_dash="dot", line_color="green",
                      annotation_text=f"目標1 {signal.target_neckline}", row=1, col=1)
        # 目標2
        fig.add_hline(y=signal.measured_move_target, line_dash="dot", line_color="darkgreen",
                      annotation_text=f"目標2 {signal.measured_move_target}", row=1, col=1)
        
        # 形態標註
        pattern_names = {
            'DOUBLE_BOTTOM': 'W底', 'DOUBLE_TOP': 'M頭',
            'HS_BOTTOM': '頭肩底', 'HS_TOP': '頭肩頂', 'CUP_HANDLE': '杯柄'
        }
        clr = 'green' if 'LONG' in signal.signal_type else 'red'
        fig.add_annotation(
            x=df.index[-1], y=df['High'].max(),
            text=f"<b>{pattern_names.get(signal.pattern_type, signal.pattern_type)}</b><br>"
                 f"品質: {signal.quality_score} | R:R 1:{signal.risk_reward_ratio}",
            showarrow=True, font=dict(color=clr, size=14),
            bgcolor='white', bordercolor=clr, borderwidth=2
        )
    
    fig.update_layout(
        title=f'{ticker} ({market}) — {tf_label}',
        xaxis_rangeslider_visible=False,
        height=550, showlegend=False, template='plotly_white'
    )
    
    if _IN_COLAB:
        fig.show(renderer='colab')
    else:
        try:
            fig.show()
        except Exception:
            display(HTML(fig.to_html(include_plotlyjs='cdn', full_html=False)))

# ============================================================
# 載入完成訊息
# ============================================================

print(f'智慧型 K 線形態辨識引擎載入完成')
print(f'   系統類型: 預判型 (Predictive)')
print(f'   ZigZag 參數: 回調門檻={ZIGZAG_THRESHOLD*100}%, 最小跨度={ZIGZAG_MIN_BARS}根')
print(f'   品質門檻: {QUALITY_THRESHOLD}分')
print(f'   支援形態: W底, M頭, 頭肩底, 頭肩頂, 杯柄')
print(f'   時框: {", ".join(TF_LABELS[t] for t in TIMEFRAMES)}')

In [ ]:
#@title 🚀 Step 4: 啟動儀表板

# ---------- 全域變數 ----------
_dm = None
_scanner = None
_results = []

# ---------- 結果儲存目錄 ----------
RESULTS_DIR = Path('./results')
RESULTS_DIR.mkdir(parents=True, exist_ok=True)

# ---------- 結果儲存函數 ----------
def save_scan_results(results: List[PatternSignal], market: str, mode: str) -> Dict[str, str]:
    """
    儲存掃描結果到檔案

    檔案格式:
    - CSV: results/scan_{market}_{date}_{time}.csv (方便 Excel 開啟)
    - JSON: results/scan_{market}_{date}_{time}.json (完整資料，含評分明細)

    Args:
        results: 掃描結果列表
        market: 市場代碼 (US/TW/ALL)
        mode: 掃描模式 (quick/full)

    Returns:
        {'csv': csv路徑, 'json': json路徑}
    """
    if not results:
        return {}

    # 產生檔名 (日期_市場_模式)
    date_str = datetime.now().strftime('%Y-%m-%d')
    time_str = datetime.now().strftime('%H%M')
    base_name = f"scan_{market}_{date_str}_{time_str}"

    csv_path = RESULTS_DIR / f"{base_name}.csv"
    json_path = RESULTS_DIR / f"{base_name}.json"

    # 1. 儲存 CSV (簡潔版，方便查看)
    df = MarketScanner.to_df(results)
    df.insert(0, '掃描時間', datetime.now().strftime('%Y-%m-%d %H:%M'))
    df.to_csv(csv_path, index=False, encoding='utf-8-sig')  # utf-8-sig 讓 Excel 正確顯示中文

    # 2. 儲存 JSON (完整版，含所有細節)
    json_data = {
        'scan_info': {
            'timestamp': datetime.now().isoformat(),
            'market': market,
            'mode': mode,
            'quality_threshold': QUALITY_THRESHOLD,
            'total_signals': len(results),
        },
        'signals': []
    }

    for r in results:
        json_data['signals'].append({
            'symbol': r.symbol,
            'pattern_type': r.pattern_type,
            'signal_type': r.signal_type,
            'timestamp': r.timestamp,
            'timeframe': r.timeframe,
            'quality_score': r.quality_score,
            'entry_zone': r.entry_zone,
            'stop_loss': r.stop_loss,
            'target_neckline': r.target_neckline,
            'measured_move_target': r.measured_move_target,
            'risk_reward_ratio': r.risk_reward_ratio,
            'score_breakdown': r.score_breakdown,
            'pattern_details': r.pattern_details,
        })

    with open(json_path, 'w', encoding='utf-8') as f:
        json.dump(json_data, f, ensure_ascii=False, indent=2)

    return {'csv': str(csv_path), 'json': str(json_path)}


def list_saved_results() -> pd.DataFrame:
    """列出所有已儲存的掃描結果"""
    files = list(RESULTS_DIR.glob('scan_*.csv'))
    if not files:
        return pd.DataFrame(columns=['檔案', '日期', '市場', '大小'])

    rows = []
    for f in sorted(files, reverse=True):
        # 解析檔名: scan_{market}_{date}_{time}.csv
        parts = f.stem.split('_')
        market = parts[1] if len(parts) > 1 else '?'
        date = parts[2] if len(parts) > 2 else '?'
        time_part = parts[3] if len(parts) > 3 else ''

        rows.append({
            '檔案': f.name,
            '日期': f"{date} {time_part[:2]}:{time_part[2:]}" if time_part else date,
            '市場': market,
            '大小': f"{f.stat().st_size / 1024:.1f} KB"
        })

    return pd.DataFrame(rows)

# ---------- UI 元件 ----------
w_fm = widgets.Text(
    value='',
    placeholder='FinMind Token (台股必填)',
    description='FinMind:',
    layout=widgets.Layout(width='520px'))

w_tg = widgets.Text(
    value='',
    placeholder='Tiingo API Key (美股主要) — 失敗自動切 yfinance',
    description='Tiingo:',
    layout=widgets.Layout(width='520px'))

w_mode = widgets.Dropdown(
    options=[
        ('快速掃描 (精選 ~120 檔)', 'quick'),
        ('全市場掃描 (全上市公司)', 'full'),
    ],
    value='quick',
    description='模式:')

w_market = widgets.Dropdown(
    options=[
        ('美股 (US)', 'US'),
        ('台股 (TW)', 'TW'),
        ('全部 (US + TW)', 'ALL')
    ],
    value='TW',
    description='市場:')

w_auto_save = widgets.Checkbox(
    value=True,
    description='自動儲存結果',
    indent=False)

w_scan = widgets.Button(
    description='開始掃描',
    button_style='success',
    layout=widgets.Layout(width='140px', height='38px'))

w_stop = widgets.Button(
    description='停止掃描',
    button_style='danger',
    disabled=True,
    layout=widgets.Layout(width='120px', height='38px'))

w_list_files = widgets.Button(
    description='查看歷史',
    button_style='info',
    layout=widgets.Layout(width='100px', height='38px'))

w_out = widgets.Output(
    layout=widgets.Layout(border='1px solid #ccc', min_height='120px'))

w_sel = widgets.Dropdown(
    options=[],
    description='查看圖表:',
    layout=widgets.Layout(width='520px'))

w_chart = widgets.Output()


# ---------- 結果表格 HTML 渲染 ----------
def _render_results_table(rdf):
    """將 DataFrame 轉為可捲動的 HTML 表格"""
    html_table = rdf.to_html(index=True, escape=False, classes='scan-results')
    return HTML(f'''
<style>
  .scroll-table {{
    max-height: 500px;
    overflow-y: auto;
    border: 1px solid #ddd;
    border-radius: 6px;
    margin: 8px 0;
  }}
  .scan-results {{
    width: 100%;
    border-collapse: collapse;
    font-size: 13px;
  }}
  .scan-results th {{
    position: sticky;
    top: 0;
    background: #2E86AB;
    color: white;
    padding: 8px 12px;
    text-align: left;
    z-index: 1;
  }}
  .scan-results td {{
    padding: 6px 12px;
    border-bottom: 1px solid #eee;
  }}
  .scan-results tr:hover {{
    background: #f0f7ff;
  }}
</style>
<div class="scroll-table">
  {html_table}
</div>
''')


# ---------- 掃描事件 ----------
def _on_scan(b):
    global _dm, _scanner, _results

    w_scan.disabled = True
    w_stop.disabled = False
    scan_ctrl.reset()

    with w_out:
        clear_output()

        tg_key   = w_tg.value.strip()
        fm_token = w_fm.value.strip()
        mode     = w_mode.value
        market   = w_market.value
        auto_save = w_auto_save.value

        # API Key 檢查
        if not tg_key and market in ('US', 'ALL'):
            print('未填 Tiingo API Key，美股將使用 yfinance 備援')

        if not fm_token and market in ('TW', 'ALL'):
            print('未填 FinMind Token，台股可能受速率限制')

        # 建立物件
        _dm = DataManager(fm_token, tg_key)
        _scanner = MarketScanner(_dm, scan_ctrl)
        _results = []

        try:
            # 取得 Ticker 清單
            print('取得掃描清單...')
            if mode == 'quick':
                tickers_us = QUICK_LIST['US'] if market in ('US','ALL') else []
                tickers_tw = QUICK_LIST['TW'] if market in ('TW','ALL') else []
            else:
                tickers_us = (TickerRegistry.fetch_us(tg_key)
                              if market in ('US','ALL') else [])
                tickers_tw = (TickerRegistry.fetch_tw(fm_token)
                              if market in ('TW','ALL') else [])

            n_us, n_tw = len(tickers_us), len(tickers_tw)
            total = n_us + n_tw
            print(f'   美股: {n_us} 檔 | 台股: {n_tw} 檔 | 合計: {total} 檔')
            print(f'   時框: {", ".join(TF_LABELS[t] for t in TIMEFRAMES)}')
            print(f'   系統: 預判型 (Predictive) | 品質門檻: {QUALITY_THRESHOLD}分')
            print()

            # 執行掃描
            if tickers_us:
                _results.extend(_scanner.scan('US', tickers_us))
            if tickers_tw and not scan_ctrl.should_stop:
                _results.extend(_scanner.scan('TW', tickers_tw))

            # 顯示結果
            print()
            if _results:
                rdf = MarketScanner.to_df(_results)
                print('=' * 55)
                print(f'結果: {len(_results)} 個預判訊號 (品質>={QUALITY_THRESHOLD})')
                print('=' * 55)
                display(_render_results_table(rdf))

                # 自動儲存結果
                if auto_save:
                    saved = save_scan_results(_results, market, mode)
                    if saved:
                        print(f'\n💾 結果已儲存:')
                        print(f'   CSV:  {saved["csv"]}')
                        print(f'   JSON: {saved["json"]}')

                # 下拉選單
                pattern_names = {
                    'DOUBLE_BOTTOM': 'W底', 'DOUBLE_TOP': 'M頭',
                    'HS_BOTTOM': '頭肩底', 'HS_TOP': '頭肩頂', 'CUP_HANDLE': '杯柄'
                }
                w_sel.options = [
                    (f"{r.symbol.split('.')[0]} | {TF_LABELS.get(r.timeframe, r.timeframe)} | "
                     f"{pattern_names.get(r.pattern_type, r.pattern_type)} | "
                     f"{r.quality_score}分 | R:R 1:{r.risk_reward_ratio}", i)
                    for i, r in enumerate(_results)
                ]
            else:
                print('未找到符合條件的預判訊號')
                if scan_ctrl.should_stop:
                    print('   (掃描已中止，僅顯示部分結果)')
                else:
                    print('   可能原因:')
                    print('   - 目前市場沒有形態正在形成中')
                    print('   - 品質門檻 65 分過濾掉了低品質訊號')
                    print('   - 預判觸發條件 (反彈/回落 2%) 尚未達成')

            # 統計摘要
            s = _dm.stats
            yf_info = f' yf備援={s["yf_fallback"]}' if s['yf_fallback'] else ''
            print(f'\n統計: API US={s["api_us"]} TW={s["api_tw"]}{yf_info} | '
                  f'快取={s["cache"]} | 錯誤={s["err"]}')

        except Exception as e:
            print(f'錯誤: {e}')
            import traceback
            traceback.print_exc()

    w_scan.disabled = False
    w_stop.disabled = True


def _on_stop(b):
    scan_ctrl.stop()
    w_stop.disabled = True
    print('正在停止，等待當前請求完成...')


def _on_list_files(b):
    """顯示歷史掃描結果"""
    with w_out:
        clear_output()
        print('📁 歷史掃描結果 (results/ 目錄)')
        print('=' * 55)
        df = list_saved_results()
        if df.empty:
            print('尚無儲存的掃描結果')
        else:
            display(df)
            print(f'\n共 {len(df)} 個檔案')
            print('提示: 可直接下載 CSV 檔案用 Excel 開啟')


def _on_select(change):
    if change['new'] is None or not _results:
        return
    r = _results[change['new']]
    ticker = r.symbol.split('.')[0]
    market = r.symbol.split('.')[1] if '.' in r.symbol else 'TW'
    with w_chart:
        clear_output()
        plot_stock(_dm, ticker, market, r)


w_scan.on_click(_on_scan)
w_stop.on_click(_on_stop)
w_list_files.on_click(_on_list_files)
w_sel.observe(_on_select, names='value')


# ---------- 顯示 Dashboard ----------
display(HTML("""
<div style='padding:12px; background:linear-gradient(135deg,#f0f7ff,#e8f4f8);
            border-radius:10px; margin-bottom:12px;'>
  <h2 style='color:#2E86AB; margin:0;'>智慧型 K 線形態辨識系統</h2>
  <p style='color:#555; margin:5px 0 0 0; font-size:13px;'>
    <b>預判型系統</b> — 在形態完成前提前識別潛在機會
  </p>
  <p style='color:#888; margin:3px 0 0 0; font-size:11px;'>
    ZigZag 3% 回調 | 品質門檻 65 分 | 支援: W底, M頭, 頭肩底, 頭肩頂, 杯柄
  </p>
  <p style='color:#888; margin:3px 0 0 0; font-size:11px;'>
    評分: 成交量 40% + 趨勢位置 30% + 形態完整度 30%
  </p>
</div>
"""))

display(widgets.VBox([
    widgets.HTML('<b>API 設定</b>'),
    w_fm, w_tg,
    widgets.HTML('<b>掃描設定</b>'),
    widgets.HBox([w_mode, w_market, w_auto_save]),
    widgets.HBox([w_scan, w_stop, w_list_files]),
], layout=widgets.Layout(
    padding='10px', border='1px solid #ddd',
    margin='0 0 10px 0', border_radius='8px')))

display(widgets.VBox([
    widgets.HTML('<b>掃描結果</b>'),
    w_out,
    w_sel,
    w_chart
], layout=widgets.Layout(
    padding='10px', border='1px solid #ddd', border_radius='8px')))

print('填入 API Key -> 選擇模式/市場 -> 點擊「開始掃描」')
print()
print('輸出欄位說明:')
print('  - 進場: 建議進場價 (當前價)')
print('  - 停損: 形態失效價 (右腳/右肩低點)')
print('  - 目標1: 頸線價格')
print('  - 目標2: 等幅測距目標')
print('  - R:R: 風險報酬比 (目標1)')
print()
print('💾 結果儲存: 勾選「自動儲存結果」後，掃描完成會自動存檔至 results/ 目錄')